# Bulk processor and improved FlySight file and jump validation

Uses `./data` as a data lake.

In [1]:
%%capture
!x=$(pip list | awk '/haversine/') ; [[ -z "$x" ]] && make local

In [2]:
from pathlib import Path

import warnings

import bokeh.models as bm
import bokeh.plotting as bp
import ipywidgets as widgets
import pandas as pd

In [3]:
from ssscoring.calc import aggregateResults
from ssscoring.calc import convertFlySight2SSScoring
from ssscoring.calc import getFlySightDataFromCSVFileName
from ssscoring.calc import isValidMaximumAltitude
from ssscoring.calc import isValidMinimumAltitude
from ssscoring.calc import processAllJumpFiles
from ssscoring.calc import processJump
from ssscoring.calc import roundedAggregateResults
from ssscoring.calc import totalResultsFrom
from ssscoring.datatypes import JumpStatus
from ssscoring.constants import FT_IN_M
from ssscoring.flysight import getAllSpeedJumpFilesFrom
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

---
## Set DZ altitude MSL

Set the value in ft.  Wingsuit World <a href='https://wingsuit.world/dropzones/' target='_blank'>List of Dropzones</a> is a good resource.

In [4]:

dropZones = {
    'Drop zone': [
        'Aerodrom Tanay',
        'Aerograd Kolomna',
        'Bay Area Skydiving',
        'Drop Zone Thailand',
        'DZ Krutitcy',
        'Fehrbellin',
        'Lodi Parachute Center',
        'Mile High',
        'Neustadt-Glewe',
        'Paracaidismo Celaya',
        'Paraclete XP',
        'Saarlouis-Düren',
        'SkyDance SkyDiving',
        'Skydive Algarve',
        'Skydive Arizona',
        'Skydive Buzz',
        'Skydive Chicago',
        'Skydive Fano',
        'Skydive Netheravon',
        'Skydive Perris',
        'Skydive Puebla',
        'Skydive Saulgau',
        'Skydive Teuge',
        'Thai Sky Adventures',
    ],
    'Alt (ft)': [
        699.0,
        472.0,
        23.0,
        15.0,
        377.0,
        138.0,
        59.0,
        5500.0,
        115.0,
        5734.0,
        304.0,
        1119.0,
        100.0,
        6.0,
        1509.0,
        840.0,
        616.0,
        52.0,
        454.0,
        1414.0,
        5744.0,
        1903.0,
        15.0,
        21.0,
    ],
}

pd.DataFrame(dropZones, columns=[ 'Drop zone', 'Alt (ft)', ])

,Drop zone,Alt (ft)
0,Aerodrom Tanay,699.0
1,Aerograd Kolomna,472.0
2,Bay Area Skydiving,23.0
3,Drop Zone Thailand,15.0
4,DZ Krutitcy,377.0
5,Fehrbellin,138.0
6,Lodi Parachute Center,59.0
7,Mile High,5500.0
8,Neustadt-Glewe,115.0
9,Paracaidismo Celaya,5734.0


In [5]:
dropZoneAltMSL = 1512.0
ignoreBaseline = True

In [6]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green">1512.00 ft</span> ( 460.86 m)<h1>')

In [7]:
jumpFiles = getAllSpeedJumpFilesFrom(Path('./data'))

In [9]:
jumpFiles

OrderedDict([(PosixPath('data/01-13-30.CSV'), '1'),
             (PosixPath('data/01-34-38.CSV'), '1'),
             (PosixPath('data/02-31-35.CSV'), '1'),
             (PosixPath('data/03-02-36.CSV'), '1'),
             (PosixPath('data/TRACK.CSV'), '2')])

---
## Process jump file


In [8]:
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda issue
jumpFilesList = list(jumpFiles.keys())
if (len(jumpFilesList) > 1):
    filePath = jumpFilesList[1]
    rawData, tag = getFlySightDataFromCSVFileName(filePath)
    data = convertFlySight2SSScoring(rawData, altitudeDZMeters=dropZoneAltMSLMeters)
    jumpResult = processJump(data)
    if jumpResult.status == JumpStatus.OK:
        display(jumpResult.table)
        display(jumpResult.window)

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
7997,5.0,171.036,74.329539,66.51,137.30,11097.134729,186.489125
8022,10.0,311.292,53.353955,80.27,222.83,9970.425989,315.831211
8047,15.0,410.652,41.342501,84.25,286.75,8298.362988,412.727837
8072,20.0,466.020,21.156027,87.40,326.89,6269.070079,466.499966
8079,21.4,470.628,21.281725,87.41,328.82,5669.923822,471.108932


PerformanceWindow(start=3507.997795903438, end=1707.0, validationStart=2713.0)

---
## Results

In [9]:
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda issue
jumpResults = processAllJumpFiles(jumpFiles=jumpFiles, altitudeDZMeters=dropZoneAltMSLMeters)
aggregate = aggregateResults(jumpResults)
aggregate

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
data 01-34-38:v1,466.17,171.0360,311.2920,410.6520,466.0200,470.6280,21.4,470.6280
data 02-31-35:v1,462.25,167.6520,295.9560,391.9320,455.5440,469.2600,22.0,469.2600
data 03-02-36:v1,468.33,174.9600,312.1560,413.0640,465.3000,473.2200,21.4,473.2200
data:v2,461.21,166.1256,294.6024,393.2748,457.1604,468.0396,22.1,468.5148


### Rounded results for training log

In [10]:
roundedResults = roundedAggregateResults(aggregate)
roundedResults

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
data 01-34-38:v1,466,171,311,411,466,471,21.4,471
data 02-31-35:v1,462,168,296,392,456,469,22.0,469
data 03-02-36:v1,468,175,312,413,465,473,21.4,473
data:v2,461,166,295,393,457,468,22.1,469


## All jumps

In [11]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)

In [12]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in jumpResults.keys():
    if ignoreBaseline and 'baseline' in resultRef:
        continue
    jumpResult = jumpResults[resultRef]
    if jumpResult.status == JumpStatus.OK:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % ('green', resultRef, 'VALID', jumpResult.score)
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - %s</span></h1>' % ('red', resultRef, 'INVALID', jumpResult.status)

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.status == JumpStatus.OK:
        belowMaxAltitude = isValidMaximumAltitude(jumpResult.data.altitudeAGL.max())
        badJumpLegend = None
        if not isValidMinimumAltitude(jumpResult.data.altitudeAGL.max()):
            badJumpLegend = '<h3><span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude AGL was lower than the minimum scoring altitude according to IPC and USPA.</h3>'
        if not belowMaxAltitude:
            badJumpLegend = '<h3><span style="color: red"><span style="font-weight: bold">RE-JUMP:</span> exit altitude AGL exceeds the maximum altitude according to IPC and USPA.</h3>'
            validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - %.02f km/h %s</span></h1>' % ('red', resultRef, 'INVALID', jumpResult.score, JumpStatus.ALTITUDE_EXCEEDS_MAXIMUM)
        display(widgets.HTML(validJumpStatus))            
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)</h3?'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if badJumpLegend:
            display(widgets.HTML(badJumpLegend))
            # TODO: Fix this logic, it's bass ackwards.
            # if not belowMaxAltitude:
            #     continue
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeAGLFt)-500.0, endY = max(jumpResult.data.altitudeAGLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips=[('Y-val', '@y{0.00}',),])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(
            individualPlot,
            jumpResult,
            lineColor=SPEED_COLORS[0])
        graphJumpResult(
            allJumpsPlot,
            jumpResult,
            lineColor=SPEED_COLORS[mixColor],
            legend='%s - %.2f' % (resultRef, jumpResult.score),
            showIt=False)

HTML(value='<hr><h1><span style="color: green">data 01-34-38:v1 jump - VALID - score = 466.17 km/h</span></h1>…

HTML(value='<h3>Max speed = 471; exit at 3507 m (11509 ft), end scoring window at 1707 m (5600 ft)</h3?')

HTML(value='<h3><span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude AGL …

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
,5.0,171,74,66.51,137.30,11097,186
,10.0,311,53,80.27,222.83,9970,316
,15.0,411,41,84.25,286.75,8298,413
,20.0,466,21,87.40,326.89,6269,466
,25.0,471,21,87.41,328.82,5670,471


HTML(value='<hr><h1><span style="color: green">data 02-31-35:v1 jump - VALID - score = 462.25 km/h</span></h1>…

...

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
,5.0,175,88,63.23,145.40,11088,196
,10.0,312,54,80.18,241.07,9943,317
,15.0,413,30,85.79,299.87,8247,414
,20.0,465,7,89.17,323.25,6213,465
,25.0,473,12,88.49,320.96,5608,473


HTML(value='<hr><h1><span style="color: green">data:v2 jump - VALID - score = 461.21 km/h</span></h1>')

HTML(value='<h3>Max speed = 469; exit at 3507 m (11506 ft), end scoring window at 1707 m (5600 ft)</h3?')

HTML(value='<h3><span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude AGL …

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
,5.0,166,58,70.84,140.21,11100,176
,10.0,295,38,82.61,172.70,10038,297
,15.0,393,52,82.40,183.27,8445,397
,20.0,457,30,86.24,180.45,6505,458
,25.0,468,34,85.87,172.75,5616,469


---
## All skydives

In [13]:
sumResults = totalResultsFrom(aggregate)
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
data 01-34-38:v1,466,171,311,411,466,471,21.4,471
data 02-31-35:v1,462,168,296,392,456,469,22.0,469
data 03-02-36:v1,468,175,312,413,465,473,21.4,473
data:v2,461,166,295,393,457,468,22.1,469


,totalSpeed,meanSpeed,meanSpeedSTD,maxScore,maxScoreSTD
totalSpeed,1857,464.25,3.304038,468,3.304038


---
## Notes from FlySight BDFL

This is the bulk of it: https://github.com/flysight/flysight-viewer-qt/blob/95442f1b3011258eed4d1ee0c4a25147a95e70ea/src/speedscoring.cpp#L172

As of the 2024 rules, the Performance Window is the part of the jump which is scored. It starts at exit and ends with 7,400 ft below exit or at Breakoff Altitude (5,600 ft AGL), whichever comes first. The score is the highest speed measured between two points 3 seconds apart, anywhere within that window, calculated using the difference in elevation/time between the two points.